In [1]:
import numpy as np

In [2]:
a = np.array([1, 2, 3.5])

In [3]:
np.isrealobj(a)

True

In [4]:
np.iscomplexobj(a)

False

In [5]:
a.real

array([1. , 2. , 3.5])

In [6]:
a.imag

array([0., 0., 0.])

In [15]:
np.random.rand(3) + np.random.rand(3)*1j

array([0.04164805+0.10154073j, 0.49932668+0.67577781j,
       0.26737182+0.98010631j])